In [1]:
import os
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, confusion_matrix
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from xgboost import XGBClassifier
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.utils.class_weight import compute_class_weight
import numpy as np
# Load merged dataset
df=pd.read_csv("road/preprocessed/merged/attack_data_without_masquerade.csv")

X = df.drop(columns=['Flag'], errors='ignore')
y = df['Flag']

# Stratified split Train/Test (70/30)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, train_size=0.7, random_state=42, stratify=y
)

# Confirm sizes
print("Train samples:", len(X_train))
print("Test samples:", len(X_test))
results = []

Train samples: 1078308
Test samples: 462133


In [5]:
"""----------DNN----------"""
#for bs in batch_sizes:
#print(f"\nTraining with batch size: 32")
bs=32
epo=5
vs=0.1
model = Sequential([   
Input(shape=(X_train.shape[1],)),  # Input layer (should be 10 features)
Dense(16, activation='relu'),#Dropout(0.3),
Dense(16, activation='relu'),#Dropout(0.3),
Dense(16, activation='relu'),#Dropout(0.3),
Dense(16, activation='relu'),#Dropout(0.3),
Dense(1, activation='sigmoid')  # Output layer: 1 neurons
])  # Rebuild model from scratch
model.compile(optimizer=Adam(), loss='binary_crossentropy', metrics=['accuracy'])

# Early stopping callback
early_stop = EarlyStopping(monitor='loss', patience=3, restore_best_weights=True)

print(f"\nTraining with batch size: {bs}")
history = model.fit(X_train, y_train, validation_split=vs, epochs=epo, batch_size=bs, verbose=1)

    # ----- Predict -----
y_pred_prob = model.predict(X_test, batch_size=1024)
y_pred = (y_pred_prob > 0.5).astype(int)

# ----- Evaluation -----
# Convert multiclass to binary: 0 = normal, 1 = any attack
y_test_binary = (y_test != 0).astype(int)
y_pred_binary = (y_pred != 0).astype(int)

# Calculate F1 score and confusion matrix
f1 = f1_score(y_test_binary, y_pred_binary, zero_division=0)
cm = confusion_matrix(y_test_binary, y_pred_binary)
tn, fp, fn, tp = cm.ravel()
# Store in your results table
dnn=model
results.append(["DNN", tn, tp, f"{f1*100:.1f}%", fn, fp])
results_df = pd.DataFrame(results, columns=["Model", "Benign Samples", "Malicious Samples", "F1 score", "FN", "FP"])
display(results_df)


Training with batch size: 32
Epoch 1/5
30328/30328 [==============================] - 111s 4ms/step - loss: 0.0857 - accuracy: 0.9791 - val_loss: 0.0426 - val_accuracy: 0.9854
Epoch 2/5
30328/30328 [==============================] - 112s 4ms/step - loss: 0.0427 - accuracy: 0.9861 - val_loss: 0.0411 - val_accuracy: 0.9852
Epoch 3/5
30328/30328 [==============================] - 116s 4ms/step - loss: 0.0401 - accuracy: 0.9870 - val_loss: 0.0418 - val_accuracy: 0.9851
Epoch 4/5
30328/30328 [==============================] - 102s 3ms/step - loss: 0.0380 - accuracy: 0.9876 - val_loss: 0.0364 - val_accuracy: 0.9879
Epoch 5/5
452/452 [==============================] - 1s 2ms/step


,Model,Benign Samples,Malicious Samples,F1 score,FN,FP
0,DNN,446538,10600,80.9%,4299,696
1,DNN,446705,10386,80.5%,4513,529
